# Instalar Libs necessarias

In [1]:
!pip install datacompy
!pip install duckdb-engine

# Autenticação

In [10]:
from google.colab import auth

auth.authenticate_user()
print('Authenticated')

from google.cloud import bigquery
from google.cloud import storage
from datetime import datetime
from io import StringIO
import numpy as np
import re
import pandas as pd
import pandas_gbq
import sys
import pytz
import datacompy


Authenticated


  warnings.warn(



In [11]:
# Selecionar o Projeto
project_id  = 'vtal-data-lake'
location_id = 'southamerica-east1'

label_id = 'user_lindineu_duran'

job_config = bigquery.QueryJobConfig()
job_config.labels = {'origem': f'{label_id}' }

#
# Seto um valor Maximo de bytes billed para evitar uma query custosa desnecessariamente
#
job_config.maximum_bytes_billed = 10485760

project = project_id   # Project ID inserted based on the query results selected to explore
location = location_id # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)

print(job_config.labels)


{'origem': 'user_lindineu_duran'}


# STEP 1 - Recuperar DF_ANT a partir de uma tabela parquet

In [12]:
import duckdb
import pandas as pd
from fsspec import filesystem
duckdb.register_filesystem(filesystem('gcs'))

nrows = 1000000
arq="gs://datalake-dev-vtal-file-cold-data/ambrosio/cre_5_2.parquet"

sql=f"""
      SELECT *
      FROM '{arq}'
      LIMIT {nrows}
      ;
"""

#sql=f"""
#      SELECT *
#      FROM '{arq}'
#  WHERE 1=1
#    AND DC = '4801070/0020'
#    AND DS in ( 'GERALDO ANTONIO REMOR' , 	'DIEGO GOMES AMORIM' )
#      ;
#"""

#sql=f"""
#      SELECT count(*)
#      FROM '{arq}'
#      ;
#"""

DF1 = duckdb.query(sql).df()
#print(DF1)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [13]:
DF1.head(5)

,MARE_CENT_CODIGO,MARE_DESCRICAO_MACRO_AREA,ESRE_DESCRICAO,DC,TIPO_PROJETO,PREX_OBSERVACAO,PSAP_PEP,CROB_STAT_CODIGO_STATUS,STAT_DESCRICAO_STATUS,CROB_DATA_INICIO_REAL,CROB_MOTIVO,DS,OPDS_DATA_INICIO_TARDE,OPDS_DATA_FINAL_TARDE
0,TLGO,TELEMONT - FIBRABR - GO,TEL CRUZEIRO DO SUL,100150000000/2001,Digitalização de cadastro de RE,AC-CZU-ECZU,,CA,Serviço Cancelado,26/02/2021,DC aberta no centro e macro rea incorretos.,CLAUDIO DOMINGOS PREGO,28/01/2021,28/01/2021
1,TLGO,TELEMONT - FIBRABR - GO,TEL CRUZEIRO DO SUL,100150000000/2001,Digitalização de cadastro de RE,AC-CZU-ECZU,,PE,Descrição Complementar Emitida,28/01/2021,,ANA FLAVIA ALVES FARIAS,28/01/2021,28/01/2021
2,TLGO,TELEMONT - FIBRABR - GO,ESTACAO RIO BRANCO CORRENTE,100150000000/2002,Digitalização de cadastro de RE,AC-RBO-RBCR,,CA,Serviço Cancelado,26/02/2021,DC aberta no centro e macro rea incorretos.,CLAUDIO DOMINGOS PREGO,28/01/2021,28/01/2021
3,TLGO,TELEMONT - FIBRABR - GO,ESTACAO RIO BRANCO CORRENTE,100150000000/2002,Digitalização de cadastro de RE,AC-RBO-RBCR,,PE,Descrição Complementar Emitida,28/01/2021,,ANA FLAVIA ALVES FARIAS,28/01/2021,28/01/2021
4,TLGO,TELEMONT - FIBRABR - GO,ESTACAO RIO BRANCO CORRENTE,100150000000/2003,Digitalização de cadastro de RE,AC-RBO-RBCR,,CA,Serviço Cancelado,26/02/2021,DC aberta no centro e macro rea incorretos.,CLAUDIO DOMINGOS PREGO,28/01/2021,28/01/2021


## Padronizar Cabecalho do DF_ANT

In [14]:
# Rename multiple columns in one go with a larger dictionary
DF1 = DF1.rename(
        columns={
                 'MARE_CENT_CODIGO'          :  'centro',
                 'MARE_DESCRICAO_MACRO_AREA' :  'macro_area',
                 'ESRE_DESCRICAO'            :  'estacao',
                 'DC'                        :  'dc',
                 'TIPO_PROJETO'              :  'tipo_projeto',
                 'PREX_OBSERVACAO'           :  'descricao',
                 'PSAP_PEP'                  :  'pep4',
                 'CROB_STAT_CODIGO_STATUS'   :  'codigo_status',
                 'STAT_DESCRICAO_STATUS'     :  'situacao',
                 'CROB_DATA_INICIO_REAL'     : 'data_situacao',
                 'CROB_MOTIVO'               : 'motivo',
                 'DS'                        : 'responsavel',
                 'OPDS_DATA_INICIO_TARDE'    : 'inicio_previsto',
                 'OPDS_DATA_FINAL_TARDE'     : 'fim_previsto'
                }
      )

In [15]:
DF1.head(5)

,centro,macro_area,estacao,dc,tipo_projeto,descricao,pep4,codigo_status,situacao,data_situacao,motivo,responsavel,inicio_previsto,fim_previsto
0,TLGO,TELEMONT - FIBRABR - GO,TEL CRUZEIRO DO SUL,100150000000/2001,Digitalização de cadastro de RE,AC-CZU-ECZU,,CA,Serviço Cancelado,26/02/2021,DC aberta no centro e macro rea incorretos.,CLAUDIO DOMINGOS PREGO,28/01/2021,28/01/2021
1,TLGO,TELEMONT - FIBRABR - GO,TEL CRUZEIRO DO SUL,100150000000/2001,Digitalização de cadastro de RE,AC-CZU-ECZU,,PE,Descrição Complementar Emitida,28/01/2021,,ANA FLAVIA ALVES FARIAS,28/01/2021,28/01/2021
2,TLGO,TELEMONT - FIBRABR - GO,ESTACAO RIO BRANCO CORRENTE,100150000000/2002,Digitalização de cadastro de RE,AC-RBO-RBCR,,CA,Serviço Cancelado,26/02/2021,DC aberta no centro e macro rea incorretos.,CLAUDIO DOMINGOS PREGO,28/01/2021,28/01/2021
3,TLGO,TELEMONT - FIBRABR - GO,ESTACAO RIO BRANCO CORRENTE,100150000000/2002,Digitalização de cadastro de RE,AC-RBO-RBCR,,PE,Descrição Complementar Emitida,28/01/2021,,ANA FLAVIA ALVES FARIAS,28/01/2021,28/01/2021
4,TLGO,TELEMONT - FIBRABR - GO,ESTACAO RIO BRANCO CORRENTE,100150000000/2003,Digitalização de cadastro de RE,AC-RBO-RBCR,,CA,Serviço Cancelado,26/02/2021,DC aberta no centro e macro rea incorretos.,CLAUDIO DOMINGOS PREGO,28/01/2021,28/01/2021


# STEP 2 - Recuperar DF_NEW a partir do BQ

In [16]:
QUERY_NEW = f"""
    SELECT * FROM `vtal-datalake-dev.trusted.tb_analitico_cre_5_2`
    #WHERE 1=1
    #AND DS_DC = '4801070/0020'
    #AND DS_RESPONSAVEL in ( 'GERALDO ANTONIO REMOR' , 	'DIEGO GOMES AMORIM' )
    LIMIT(1000000)
;
"""


In [17]:
DF2 = pandas_gbq.read_gbq(QUERY_NEW, project_id=project_id, configuration = {'labels': {'origem':f'label_id'} } )
print( f"Tamanho DF1 [{len(DF1)}]  DF2 [{len(DF2)}] " )

Downloading: 100%|██████████|
Tamanho DF1 [1000000]  DF2 [1000000] 


## Padronizar Cabecalho do DF_NEW

In [18]:
# Rename multiple columns in one go with a larger dictionary
DF2 = DF2.rename(
        columns={
                 'ds_centro'                  :  'centro',
                 'ds_macro_area'              :  'macro_area',
                 'ds_estacao'                 :  'estacao',
                 'ds_dc'                      :  'dc',
                 'ds_tipo_projeto'            :  'tipo_projeto',
                 'ds_descricao'               :  'descricao',
                 'ds_pep4'                    :  'pep4',
                 'ds_crob_stat_codigo_status' :  'codigo_status',
                 'ds_situacao'                :  'situacao',
                 'dh_data_situacao'           : 'data_situacao',
                 'ds_motivo'                  : 'motivo',
                 'ds_responsavel'             : 'responsavel',
                 'nu_qt_dias'                 : 'qt_dias',
                 'dh_inicio_previsto'         : 'inicio_previsto',
                 'dh_fim_previsto'            : 'fim_previsto'
                }
      )

## Remover as colunas que so existem na Origem

In [19]:
DF1.columns

Index(['centro', 'macro_area', 'estacao', 'dc', 'tipo_projeto', 'descricao',
       'pep4', 'codigo_status', 'situacao', 'data_situacao', 'motivo',
       'responsavel', 'inicio_previsto', 'fim_previsto'],
      dtype='object')

In [20]:
DF2.columns

Index(['centro', 'macro_area', 'estacao', 'dc', 'tipo_projeto', 'descricao',
       'pep4', 'codigo_status', 'situacao', 'data_situacao', 'motivo',
       'responsavel', 'qt_dias', 'inicio_previsto', 'fim_previsto'],
      dtype='object')

ATENCAO: Se retornar coluna diferente em DF2 tem q ser analisado !!!!!!

In [21]:
# Check para verificar se realmente os DataFrames possuem as mesmas colunas

exists_only_in_df1 = set(DF1.columns) - set(DF2.columns)
exists_only_in_df2 = set(DF2.columns) - set(DF1.columns)

print("Diferenças entre DF1 e DF2:")
for col in DF1.columns:
    if col in exists_only_in_df1:
        print(f"{col} existe somente em DF1")
    elif col in exists_only_in_df2:
        print(f"{col} existe somente em DF2")

for col in DF2.columns:
    if col in exists_only_in_df2:
        print(f"{col} existe somente em DF2")

Diferenças entre DF1 e DF2:
qt_dias existe somente em DF2


Remover as colunas da origem ( DF1 ) que nao existem no Destino ( DF2 )

In [22]:
for col in exists_only_in_df1:
    DF1 = DF1.drop(col, axis=1)

## Equalizar o campo de data

In [23]:
#DF2['repr_data_producao'] = DF2['repr_data_producao'].dt.date
DF1['data_situacao'] = pd.to_datetime(DF1['data_situacao']).dt.tz_localize('UTC')
DF1['inicio_previsto'] = pd.to_datetime(DF1['inicio_previsto']).dt.tz_localize('UTC')
DF1['fim_previsto'] = pd.to_datetime(DF1['fim_previsto']).dt.tz_localize('UTC')


  DF1['data_situacao'] = pd.to_datetime(DF1['data_situacao']).dt.tz_localize('UTC')

  DF1['inicio_previsto'] = pd.to_datetime(DF1['inicio_previsto']).dt.tz_localize('UTC')



OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 19/04/0021, at position 2. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

## Aplicar upper nas colunas da Origem ( DF1 )  para equalizar com o destino

In [24]:
columns_to_upper = ['tipo_projeto', 'situacao', 'motivo']
for column in columns_to_upper:
    DF1[column] = DF1[column].str.upper()
print(DF1)

       centro               macro_area                               estacao  \
0        TLGO  TELEMONT - FIBRABR - GO                   TEL CRUZEIRO DO SUL   
1        TLGO  TELEMONT - FIBRABR - GO                   TEL CRUZEIRO DO SUL   
2        TLGO  TELEMONT - FIBRABR - GO           ESTACAO RIO BRANCO CORRENTE   
3        TLGO  TELEMONT - FIBRABR - GO           ESTACAO RIO BRANCO CORRENTE   
4        TLGO  TELEMONT - FIBRABR - GO           ESTACAO RIO BRANCO CORRENTE   
...       ...                      ...                                   ...   
999995   TLPR     SEREDE - PR INTERIOR  COL WITMARSUM AV PRES ERNESTO GEISEL   
999996   TLPR     SEREDE - PR INTERIOR  COL WITMARSUM AV PRES ERNESTO GEISEL   
999997   TLPR     SEREDE - PR INTERIOR  COL WITMARSUM AV PRES ERNESTO GEISEL   
999998   TLPR     SEREDE - PR INTERIOR  COL WITMARSUM AV PRES ERNESTO GEISEL   
999999   TLPR     SEREDE - PR INTERIOR  COL WITMARSUM AV PRES ERNESTO GEISEL   

                       dc              

# STEP 4 - Comparar os Dataframes ( origem x destino)

In [25]:
compare = datacompy.Compare(
    DF1,
    DF2,
    join_columns=["dc","responsavel" ],
    df1_name='ANT',
    df2_name='NEW'
    )
compare.matches(ignore_extra_columns=False)
#print(compare.report(sample_count=20))
print(compare.report(sample_count=100))

  (pd.to_datetime(obj_column) == date_column)

  (pd.to_datetime(obj_column) == date_column)



DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns     Rows
0       ANT       14  1000000
1       NEW       15  1000000

Column Summary
--------------

Number of columns in common: 14
Number of columns in ANT but not in NEW: 0 []
Number of columns in NEW but not in ANT: 1 ['qt_dias']

Row Summary
-----------

Matched on: dc, responsavel
Any duplicates on match values: Yes
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 23,157
Number of rows in ANT but not in NEW: 976,843
Number of rows in NEW but not in ANT: 976,843

Number of rows with some compared columns unequal: 23,157
Number of rows with all compared columns equal: 0

Column Comparison
-----------------

Number of columns compared with some values unequal: 8
Number of columns compared with all values equal: 6
Total number of values which compare unequal: 108,230

Columns with Unequal Values or Types
------------------------------------

            Column    

Foram comparadas duas extracoes full e como todas as colunas estao identicas, nao foram efetuados os testes nas TBs ...
Utilizado sample_count=100 para comparar a tabela inteira ...